# Probability Exercises for AI

This notebook provides hands-on exercises with probability concepts essential for machine learning.

## Learning Objectives
- Work with probability distributions
- Apply Bayes' theorem to real problems
- Perform Monte Carlo simulations
- Visualize probability concepts

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import pandas as pd

# Set style for better visualizations
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
np.random.seed(42)

## 1. Probability Distributions

### 1.1 Bernoulli Distribution
Models binary outcomes (success/failure)

In [ ]:
# Bernoulli distribution: Coin flip
p = 0.6  # Probability of heads

# Simulate 1000 coin flips
flips = np.random.binomial(1, p, 1000)

print(f"Theoretical probability of heads: {p}")
print(f"Empirical probability of heads: {flips.mean():.3f}")
print(f"Number of heads: {flips.sum()}")
print(f"Number of tails: {len(flips) - flips.sum()}")

# Visualize
plt.figure(figsize=(8, 5))
plt.bar(['Tails (0)', 'Heads (1)'], [len(flips) - flips.sum(), flips.sum()], color=['red', 'blue'])
plt.title('Bernoulli Distribution: Coin Flips (p=0.6)')
plt.ylabel('Count')
plt.axhline(y=500, color='gray', linestyle='--', label='Expected (p=0.5)')
plt.legend()
plt.show()

### 1.2 Binomial Distribution
Number of successes in n independent trials

In [ ]:
# Binomial: Number of heads in 10 coin flips
n_trials = 10
p_success = 0.5
n_experiments = 10000

# Simulate
results = np.random.binomial(n_trials, p_success, n_experiments)

# Theoretical distribution
x = np.arange(0, n_trials + 1)
theoretical_pmf = stats.binom.pmf(x, n_trials, p_success)

# Plot
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(results, bins=np.arange(0, n_trials + 2) - 0.5, density=True, alpha=0.7, color='blue', edgecolor='black')
plt.plot(x, theoretical_pmf, 'ro-', linewidth=2, label='Theoretical')
plt.xlabel('Number of Heads')
plt.ylabel('Probability')
plt.title(f'Binomial Distribution (n={n_trials}, p={p_success})')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
cumulative = np.cumsum(theoretical_pmf)
plt.plot(x, cumulative, 'go-', linewidth=2)
plt.xlabel('Number of Heads')
plt.ylabel('Cumulative Probability')
plt.title('Cumulative Distribution Function (CDF)')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Mean (theoretical): {n_trials * p_success}")
print(f"Mean (empirical): {results.mean():.3f}")
print(f"Variance (theoretical): {n_trials * p_success * (1 - p_success)}")
print(f"Variance (empirical): {results.var():.3f}")

### 1.3 Normal (Gaussian) Distribution
The most important distribution in statistics and ML

In [ ]:
# Normal distribution parameters
mu = 0  # mean
sigma = 1  # standard deviation

# Generate samples
samples = np.random.normal(mu, sigma, 10000)

# Theoretical distribution
x = np.linspace(-4, 4, 100)
pdf = stats.norm.pdf(x, mu, sigma)

# Plot
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.hist(samples, bins=50, density=True, alpha=0.7, color='skyblue', edgecolor='black')
plt.plot(x, pdf, 'r-', linewidth=2, label='Theoretical PDF')
plt.axvline(mu, color='green', linestyle='--', linewidth=2, label=f'Mean = {mu}')
plt.axvline(mu - sigma, color='orange', linestyle='--', label=f'±1σ')
plt.axvline(mu + sigma, color='orange', linestyle='--')
plt.xlabel('Value')
plt.ylabel('Probability Density')
plt.title(f'Normal Distribution (μ={mu}, σ={sigma})')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
stats.probplot(samples, dist="norm", plot=plt)
plt.title('Q-Q Plot (Normal Distribution Check)')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 68-95-99.7 rule
within_1_sigma = np.sum((samples >= mu - sigma) & (samples <= mu + sigma)) / len(samples)
within_2_sigma = np.sum((samples >= mu - 2*sigma) & (samples <= mu + 2*sigma)) / len(samples)
within_3_sigma = np.sum((samples >= mu - 3*sigma) & (samples <= mu + 3*sigma)) / len(samples)

print("68-95-99.7 Rule Verification:")
print(f"Within 1σ: {within_1_sigma:.1%} (expected: 68%)")
print(f"Within 2σ: {within_2_sigma:.1%} (expected: 95%)")
print(f"Within 3σ: {within_3_sigma:.1%} (expected: 99.7%)")

### 1.4 Comparing Multiple Distributions

In [ ]:
# Compare different normal distributions
x = np.linspace(-10, 10, 1000)

distributions = [
    {'mu': 0, 'sigma': 1, 'label': 'μ=0, σ=1', 'color': 'blue'},
    {'mu': 0, 'sigma': 2, 'label': 'μ=0, σ=2', 'color': 'red'},
    {'mu': 2, 'sigma': 1, 'label': 'μ=2, σ=1', 'color': 'green'},
    {'mu': -2, 'sigma': 0.5, 'label': 'μ=-2, σ=0.5', 'color': 'orange'}
]

plt.figure(figsize=(12, 6))
for dist in distributions:
    pdf = stats.norm.pdf(x, dist['mu'], dist['sigma'])
    plt.plot(x, pdf, label=dist['label'], color=dist['color'], linewidth=2)

plt.xlabel('Value')
plt.ylabel('Probability Density')
plt.title('Comparison of Normal Distributions')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 2. Bayes' Theorem

### 2.1 Medical Test Example
Classic example: Disease diagnosis with imperfect test

In [ ]:
# Problem setup
P_disease = 0.01  # 1% of population has disease
P_positive_given_disease = 0.99  # Test sensitivity (true positive rate)
P_positive_given_no_disease = 0.05  # False positive rate

# Calculate P(positive)
P_no_disease = 1 - P_disease
P_positive = (P_positive_given_disease * P_disease + 
              P_positive_given_no_disease * P_no_disease)

# Bayes' theorem: P(disease | positive)
P_disease_given_positive = (P_positive_given_disease * P_disease) / P_positive

print("Medical Test Analysis")
print("=" * 50)
print(f"Disease prevalence: {P_disease:.1%}")
print(f"Test sensitivity (true positive): {P_positive_given_disease:.1%}")
print(f"False positive rate: {P_positive_given_no_disease:.1%}")
print(f"\nP(positive test): {P_positive:.2%}")
print(f"\nP(disease | positive test): {P_disease_given_positive:.2%}")
print(f"\n⚠️ Despite 99% accurate test, only {P_disease_given_positive:.1%} chance of having disease!")

# Visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Population breakdown
population = 10000
diseased = int(population * P_disease)
healthy = population - diseased
true_positive = int(diseased * P_positive_given_disease)
false_positive = int(healthy * P_positive_given_no_disease)

ax1.bar(['Diseased', 'Healthy'], [diseased, healthy], color=['red', 'green'])
ax1.set_ylabel('Number of People')
ax1.set_title(f'Population of {population:,} People')
ax1.grid(True, alpha=0.3)

# Test results
categories = ['True\nPositive', 'False\nPositive', 'True\nNegative', 'False\nNegative']
values = [true_positive, false_positive, healthy - false_positive, diseased - true_positive]
colors = ['darkgreen', 'orange', 'lightgreen', 'darkred']

ax2.bar(categories, values, color=colors)
ax2.set_ylabel('Number of People')
ax2.set_title('Test Results Breakdown')
ax2.grid(True, alpha=0.3)

for i, v in enumerate(values):
    ax2.text(i, v + 20, str(v), ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

print(f"\nOut of {true_positive + false_positive} positive tests:")
print(f"  - True positives: {true_positive}")
print(f"  - False positives: {false_positive}")
print(f"  - Probability of disease: {true_positive/(true_positive + false_positive):.1%}")

### 2.2 Spam Filter Example
Naive Bayes classifier for spam detection

In [ ]:
# Simplified spam filter
P_spam = 0.3  # 30% of emails are spam

# Word probabilities
words = ['free', 'money', 'meeting', 'lunch']
P_word_given_spam = [0.8, 0.7, 0.1, 0.05]
P_word_given_ham = [0.1, 0.05, 0.6, 0.4]

# Email contains: "free money"
email_words = ['free', 'money']

# Calculate P(spam | words)
P_ham = 1 - P_spam

# Naive assumption: words are independent
P_words_given_spam = 1
P_words_given_ham = 1

for word in email_words:
    idx = words.index(word)
    P_words_given_spam *= P_word_given_spam[idx]
    P_words_given_ham *= P_word_given_ham[idx]

# Bayes' theorem
numerator_spam = P_words_given_spam * P_spam
numerator_ham = P_words_given_ham * P_ham
denominator = numerator_spam + numerator_ham

P_spam_given_words = numerator_spam / denominator
P_ham_given_words = numerator_ham / denominator

print("Spam Filter Analysis")
print("=" * 50)
print(f"Email contains: {', '.join(email_words)}")
print(f"\nP(spam): {P_spam:.1%}")
print(f"P(ham): {P_ham:.1%}")
print(f"\nP(words | spam): {P_words_given_spam:.3f}")
print(f"P(words | ham): {P_words_given_ham:.3f}")
print(f"\nP(spam | words): {P_spam_given_words:.1%}")
print(f"P(ham | words): {P_ham_given_words:.1%}")
print(f"\n📧 Classification: {'SPAM' if P_spam_given_words > 0.5 else 'HAM'}")

# Visualization
plt.figure(figsize=(10, 5))
plt.bar(['Spam', 'Ham'], [P_spam_given_words, P_ham_given_words], color=['red', 'green'])
plt.ylabel('Probability')
plt.title(f'Email Classification: "{" ".join(email_words)}"')
plt.ylim(0, 1)
plt.axhline(y=0.5, color='black', linestyle='--', label='Decision Threshold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 3. Monte Carlo Simulations

### 3.1 Estimating π using Monte Carlo

In [ ]:
def estimate_pi(n_samples):
    """Estimate π by randomly sampling points in a square"""
    # Generate random points in [0, 1] x [0, 1]
    x = np.random.uniform(0, 1, n_samples)
    y = np.random.uniform(0, 1, n_samples)
    
    # Check if points are inside quarter circle
    inside_circle = (x**2 + y**2) <= 1
    
    # π ≈ 4 * (points inside circle / total points)
    pi_estimate = 4 * np.sum(inside_circle) / n_samples
    
    return pi_estimate, x, y, inside_circle

# Run simulation
n_samples = 10000
pi_estimate, x, y, inside = estimate_pi(n_samples)

print(f"Estimated π: {pi_estimate:.6f}")
print(f"Actual π: {np.pi:.6f}")
print(f"Error: {abs(pi_estimate - np.pi):.6f}")

# Visualization
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.scatter(x[inside], y[inside], c='red', s=1, alpha=0.5, label='Inside circle')
plt.scatter(x[~inside], y[~inside], c='blue', s=1, alpha=0.5, label='Outside circle')
circle = plt.Circle((0, 0), 1, fill=False, color='black', linewidth=2)
plt.gca().add_patch(circle)
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.gca().set_aspect('equal')
plt.title(f'Monte Carlo Estimation of π\n{n_samples:,} samples')
plt.legend()
plt.grid(True, alpha=0.3)

# Convergence plot
plt.subplot(1, 2, 2)
sample_sizes = np.logspace(2, 5, 50, dtype=int)
estimates = [estimate_pi(n)[0] for n in sample_sizes]

plt.semilogx(sample_sizes, estimates, 'b-', linewidth=2, label='Estimate')
plt.axhline(y=np.pi, color='red', linestyle='--', linewidth=2, label='True π')
plt.xlabel('Number of Samples')
plt.ylabel('Estimated π')
plt.title('Convergence to π')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 3.2 Central Limit Theorem Demonstration

In [ ]:
# Central Limit Theorem: Sample means approach normal distribution
n_samples = 1000
sample_sizes = [1, 5, 10, 30]

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.ravel()

# Original distribution: Uniform [0, 1]
for idx, n in enumerate(sample_sizes):
    # Generate samples and compute means
    sample_means = []
    for _ in range(n_samples):
        sample = np.random.uniform(0, 1, n)
        sample_means.append(sample.mean())
    
    sample_means = np.array(sample_means)
    
    # Plot histogram
    axes[idx].hist(sample_means, bins=30, density=True, alpha=0.7, color='skyblue', edgecolor='black')
    
    # Overlay normal distribution
    mu = 0.5  # Mean of uniform [0, 1]
    sigma = 1 / np.sqrt(12 * n)  # Standard error
    x = np.linspace(sample_means.min(), sample_means.max(), 100)
    pdf = stats.norm.pdf(x, mu, sigma)
    axes[idx].plot(x, pdf, 'r-', linewidth=2, label='Normal PDF')
    
    axes[idx].set_xlabel('Sample Mean')
    axes[idx].set_ylabel('Density')
    axes[idx].set_title(f'Sample Size n = {n}')
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)

plt.suptitle('Central Limit Theorem: Distribution of Sample Means', fontsize=16, y=1.00)
plt.tight_layout()
plt.show()

print("Central Limit Theorem:")
print("As sample size increases, the distribution of sample means")
print("approaches a normal distribution, regardless of the original distribution!")

## 4. Expected Value and Variance

### 4.1 Calculating Expected Value

In [ ]:
# Dice roll example
outcomes = np.array([1, 2, 3, 4, 5, 6])
probabilities = np.array([1/6, 1/6, 1/6, 1/6, 1/6, 1/6])

# Expected value: E[X] = Σ x * P(x)
expected_value = np.sum(outcomes * probabilities)

# Variance: Var(X) = E[X²] - (E[X])²
expected_x_squared = np.sum(outcomes**2 * probabilities)
variance = expected_x_squared - expected_value**2
std_dev = np.sqrt(variance)

print("Fair Dice Statistics")
print("=" * 50)
print(f"Expected value: {expected_value:.3f}")
print(f"Variance: {variance:.3f}")
print(f"Standard deviation: {std_dev:.3f}")

# Simulate and verify
n_rolls = 100000
rolls = np.random.choice(outcomes, size=n_rolls, p=probabilities)

print(f"\nEmpirical (from {n_rolls:,} rolls):")
print(f"Mean: {rolls.mean():.3f}")
print(f"Variance: {rolls.var():.3f}")
print(f"Std dev: {rolls.std():.3f}")

# Visualization
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.bar(outcomes, probabilities, color='steelblue', edgecolor='black')
plt.axvline(expected_value, color='red', linestyle='--', linewidth=2, label=f'E[X] = {expected_value:.2f}')
plt.xlabel('Outcome')
plt.ylabel('Probability')
plt.title('Probability Mass Function')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.hist(rolls, bins=np.arange(0.5, 7.5, 1), density=True, alpha=0.7, color='lightblue', edgecolor='black')
plt.axvline(rolls.mean(), color='red', linestyle='--', linewidth=2, label=f'Sample Mean = {rolls.mean():.2f}')
plt.xlabel('Outcome')
plt.ylabel('Relative Frequency')
plt.title(f'Empirical Distribution ({n_rolls:,} rolls)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Practical ML Application: Probability in Classification

### 5.1 Softmax Function

In [ ]:
def softmax(logits):
    """Convert logits to probabilities"""
    exp_logits = np.exp(logits - np.max(logits))  # Numerical stability
    return exp_logits / exp_logits.sum()

# Example: Image classification logits
logits = np.array([2.0, 1.0, 0.1])  # Raw scores from neural network
class_names = ['Cat', 'Dog', 'Bird']

probabilities = softmax(logits)

print("Image Classification Example")
print("=" * 50)
print("Logits (raw scores):")
for name, logit in zip(class_names, logits):
    print(f"  {name}: {logit:.2f}")

print("\nProbabilities (after softmax):")
for name, prob in zip(class_names, probabilities):
    print(f"  {name}: {prob:.1%}")

print(f"\nSum of probabilities: {probabilities.sum():.6f}")
print(f"Predicted class: {class_names[np.argmax(probabilities)]}")

# Visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.bar(class_names, logits, color='steelblue', edgecolor='black')
ax1.set_ylabel('Logit Value')
ax1.set_title('Raw Logits from Neural Network')
ax1.grid(True, alpha=0.3)

colors = ['green' if p == probabilities.max() else 'lightblue' for p in probabilities]
ax2.bar(class_names, probabilities, color=colors, edgecolor='black')
ax2.set_ylabel('Probability')
ax2.set_title('Probabilities after Softmax')
ax2.set_ylim(0, 1)
ax2.grid(True, alpha=0.3)

for i, (name, prob) in enumerate(zip(class_names, probabilities)):
    ax2.text(i, prob + 0.02, f'{prob:.1%}', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

## Summary

In this notebook, we covered:

1. **Probability Distributions**
   - Bernoulli, Binomial, and Normal distributions
   - Empirical vs theoretical distributions

2. **Bayes' Theorem**
   - Medical diagnosis example
   - Spam filtering application

3. **Monte Carlo Simulations**
   - Estimating π
   - Central Limit Theorem

4. **Expected Value and Variance**
   - Theoretical calculations
   - Empirical verification

5. **ML Applications**
   - Softmax function for classification

## Key Takeaways

- Probability is fundamental to machine learning
- Bayes' theorem enables probabilistic reasoning
- Monte Carlo methods approximate complex distributions
- Understanding distributions helps interpret ML model outputs

## Next Steps

Continue to [Linear Algebra Operations](./02_linear_algebra_operations.ipynb)